In [1]:
import json
import openai

In [2]:
openai.api_key = "xxx"

In [3]:
class Example():
    """Stores an input, output pair and formats it to prime the model."""

    def __init__(self, inp, out):
        self.input = inp
        self.output = out

    def get_input(self):
        """Returns the input of the example."""
        return self.input

    def get_output(self):
        """Returns the intended output of the example."""
        return self.output

    def format(self):
        """Formats the input, output pair."""
        return f"input: {self.input}\noutput: {self.output}\n"


class GPT:
    """The main class for a user to interface with the OpenAI API.
    A user can add examples and set parameters of the API request."""

    def __init__(self, engine='davinci',
                 temperature=0.5,
                 max_tokens=100):
        self.examples = []
        self.engine = engine
        self.temperature = temperature
        self.max_tokens = max_tokens

    def add_example(self, ex):
        """Adds an example to the object. Example must be an instance
        of the Example class."""
        assert isinstance(ex, Example), "Please create an Example object."
        self.examples.append(ex.format())

    def get_prime_text(self):
        """Formats all examples to prime the model."""
        return '\n'.join(self.examples) + '\n'

    def get_engine(self):
        """Returns the engine specified for the API."""
        return self.engine

    def get_temperature(self):
        """Returns the temperature specified for the API."""
        return self.temperature

    def get_max_tokens(self):
        """Returns the max tokens specified for the API."""
        return self.max_tokens

    def craft_query(self, prompt):
        """Creates the query for the API request."""
        return self.get_prime_text() + "input: " + prompt + "\n"

    def submit_request(self, prompt):
        """Calls the OpenAI API with the specified parameters."""
        response = openai.Completion.create(engine=self.get_engine(),
                                            prompt=self.craft_query(prompt),
                                            max_tokens=self.get_max_tokens(),
                                            temperature=self.get_temperature(),
                                            top_p=1,
                                            n=1,
                                            stream=False,
                                            stop="\ninput:")
        return response

    def get_top_reply(self, prompt):
        """Obtains the best result as returned by the API."""
        response = self.submit_request(prompt)
        return response['choices'][0]['text'].split('output:')[1]

In [4]:
gpt = GPT(engine="davinci",
          temperature=0.9,
          max_tokens=400)

In [5]:
class EmploymentInfo:
    def __init__(self, company_name, boss_name, employee_name, last_raise, job_title, achievements, raise_proposal, years_employed=None):
        self.company_name = company_name
        self.boss_name = boss_name
        self.employee_name = employee_name
        self.last_raise = last_raise
        self.job_title = job_title
        self.achievements = achievements
        self.raise_proposal = raise_proposal
        self.years_employed = years_employed
        
    def __str__(self):
        str_achievements = '|'.join(self.achievements)
        content = f'{self.company_name}\n{self.boss_name}\n{self.employee_name}\n{self.last_raise}\n{self.job_title}\n{str_achievements}\n{self.raise_proposal}'
        if self.years_employed is not None:
            content += f'\n{self.years_employed}'
        return content
    
def getPayRiseLetterContent(employment_info, gpt):
    prompt = str(employment_info)
    resp = gpt.get_top_reply(prompt)
    return f'Dear {employment_info.boss_name},\n{resp}\n{employment_info.employee_name}'

In [6]:
employment_info_1 = EmploymentInfo(company_name='Chandlers', boss_name='Robert', employee_name='Kate', last_raise='4 years', job_title='software engineer', achievements=['took initiative to redesign the web app', 'developed a machine learning model to automate anomaly detection'], raise_proposal='10%')
letter1 = """I have greatly enjoyed working at Chandlers for the past four years. In those years, I have become an integral member of the software engineering team, and have developed innovative ways to contribute to the company.
For example, in the past year alone, I have taken the intiative to redesign the web app and have developed a machine learning model to automate anomaly detection.
I believe I have gone above and beyond the benchmarks we set for my position in the last four years.
I would therefore appreciate the opportunity to meet with you to discuss increasing my salary so that it is commensurate with my current performance. I request a pay raise of 10%, which I believe reflects both my current competencies and industry averages.
Once again, I am grateful to be a member of this organization, and I enjoy taking on assignments that allow me to contribute to the company.
Thank you for your time. I look forward to speaking with you soon.
Sincerely,
"""
employment_info_2 = EmploymentInfo(company_name='ABC Corp', boss_name='Jane', employee_name='George Smith', last_raise='2 years ago', job_title='UX Designer', achievements=['completed XYZ project', 'exceeded my own goals without missing a single deadline', 'continued developing my skillset', 'taking classes in UX design'], raise_proposal='8%', years_employed=4)
letter2 = """Now that the XYZ project is in the rear view and we’re all settling back into our regular routines, I wanted to drop you a line to ask if we can have a meeting to discuss my compensation.
As you know, I started at ABC Corp four years ago as an intern and came on board at a salary that was slightly low in the pay band, with the understanding that we would revisit my pay at review time. Since then, of course, we’ve all been too busy to think much about anything but hitting our deadlines.
I feel so fortunate to have had the opportunity to start my career with a mentor like you, and to continue to learn in a company that’s growing so fast. Over the past two years, I’ve gladly assumed many hats, including completing XYz project. In addition, I’ve always exceeded my own goals without missing a single deadline. I’ve also continued developing my skillset, taking classes in UX design.
My research indicates that a raise of 8% would be appropriate. I’d love the opportunity to meet with you and discuss in person.
Best,"""
employment_info_3 = EmploymentInfo(company_name='SalesCorp', boss_name='Robert', employee_name='Phil', last_raise='last year', job_title='key accountant', achievements=['took the lead on expenses optimization', 'met goals in reducing the rental cost', 'improved efficiencies that led to a savings of $5000 for the company', 'achieved success in mentoring Jane'], raise_proposal='8%', years_employed=5)
letter3 = """As the fifth anniversary gets close, I would like to formally request a review of my salary for my work as a key accountant. During my time at SalesCorp, I have taken on additional responsibilities and have achieved success in several areas. I’ve made a brief list of just some of my accomplishments and responsibilities, which include the following:
Taking the lead on expenses optimization
Meeting goals in reducing the rental cost
Improving efficiencies that led to a savings of $5000 for the company
Successfully mentored Jane
I enjoy my work here and appreciate the support you and the team have shown me. Given the added value I have brought the company, I think it is fair to request a bump in pay. Based on the research, I’ve found that the midpoint salary (or median national salary) for this position is slightly higher for my experience level. Considering regional variances, a 8 percent raise would put my compensation closer to those salary benchmarks.
Thank you for your consideration. I am willing to work with you to accommodate my request, taking into account what is best for the company. Please let me know if you are available for a short meeting.
Sincerely,"""


In [7]:
gpt.add_example(Example(employment_info_1, 
                        letter1))

In [8]:
gpt.add_example(Example(employment_info_2, 
                        letter2))

In [9]:
gpt.add_example(Example(employment_info_3, 
                        letter3))

In [10]:
employment_info = EmploymentInfo(company_name='Cognos Softworks', boss_name="Steve", employee_name="Aaron", last_raise='last year', job_title='project manager', achievements=['lead the development of the LIDAR', 'my team exceeded the expectation under my leadership', 'introduced SCRUM'], raise_proposal='12%')
print(getPayRiseLetterContent(employment_info, gpt))

Dear Steve,
 I’ve greatly enjoyed my time here at Cognos Softworks. As a project manager, my role at the company has been extremely rewarding, and I know that I have made serious contributions in ways that you can’t measure on paper. I’d like to take this opportunity to submit my formal request for a review of my wage to reflect my experience and the value I’ve brought to the company.
As you know, I lead the development of the LIDAR last year. My team exceeded the expectation under my leadership, resulting in a product that was both performant and provided a recognizable return on investment. I also introduced SCRUM , which allowed us to complete the product in half the time and using fewer team members than we had originally anticipated. This means that we not only hit our deadline, but we also generated an average of 15 percent more business per quarter, compared to financial forecasts before we began working in that way.
Thanks for your consideration. I would love to discuss this mo

# References
[gpt3-sandbox](https://github.com/shreyashankar/gpt3-sandbox/blob/master/api/gpt.py)